<a href="https://colab.research.google.com/github/ayushanand960/Drowsiness_Detection_System/blob/main/Drowsiness_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup Environment

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Import Libraries


In [2]:
!unzip /content/drive/MyDrive/MRL_Eye_Dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: open_eye/s0024_00466_0_1_1_0_0_01.png  
  inflating: open_eye/s0024_00467_0_1_1_0_0_01.png  
  inflating: open_eye/s0024_00468_0_1_1_0_0_01.png  
  inflating: open_eye/s0024_00469_0_1_1_2_0_01.png  
  inflating: open_eye/s0024_00470_0_1_1_0_0_01.png  
  inflating: open_eye/s0024_00471_0_1_1_0_0_01.png  
  inflating: open_eye/s0024_00472_0_1_1_0_0_01.png  
  inflating: open_eye/s0024_00473_0_1_1_0_0_01.png  
  inflating: open_eye/s0024_00474_0_1_1_0_0_01.png  
  inflating: open_eye/s0024_00475_0_1_1_0_0_01.png  
  inflating: open_eye/s0024_00476_0_1_1_0_0_01.png  
  inflating: open_eye/s0024_00477_0_1_1_0_0_01.png  
  inflating: open_eye/s0024_00478_0_1_1_0_0_01.png  
  inflating: open_eye/s0024_00479_0_1_1_0_0_01.png  
  inflating: open_eye/s0024_00480_0_1_1_1_0_01.png  
  inflating: open_eye/s0024_00481_0_1_1_0_0_01.png  
  inflating: open_eye/s0024_00482_0_1_1_0_0_01.png  
  inflating: open_eye/s0024_00483_0_1_1_0_0_01.p

In [13]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


# Dataset Preparation


In [14]:
dataset_path = '/content/MRL_Dataset'
if not os.path.exists(dataset_path):
    print(f"Dataset path '{dataset_path}' does not exist. Please check and upload your dataset.")
else:
    print(f"Dataset found at: {dataset_path}")

Dataset found at: /content/MRL_Dataset


In [15]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize to 128x128
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize (assuming grayscale images)
])

In [16]:
dataset = datasets.ImageFolder(dataset_path, transform=transform)

In [17]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

In [18]:
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"Dataset loaded with {len(train_dataset)} training samples and {len(test_dataset)} testing samples.")

Dataset loaded with 38400 training samples and 9600 testing samples.


## Model Building

In [19]:
class SleepAwakeCNN(nn.Module):
    def __init__(self):
        super(SleepAwakeCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),  # Conv layer 1
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Downsampling

            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),  # Conv layer 2
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),  # Conv layer 3
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 16 * 16, 512),  # Fully connected layer 1
            nn.ReLU(),
            nn.Dropout(0.5),  # Dropout for regularization
            nn.Linear(512, 2)  # Output layer (2 classes: awake/asleep)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x


In [20]:
model = SleepAwakeCNN().to(device)

In [21]:
criterion = nn.CrossEntropyLoss()  # Loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Optimizer

In [22]:
num_epochs = 15
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Zero gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader):.4f}")

Epoch 1/15, Loss: 0.1689
Epoch 2/15, Loss: 0.0733
Epoch 3/15, Loss: 0.0454
Epoch 4/15, Loss: 0.0344
Epoch 5/15, Loss: 0.0276
Epoch 6/15, Loss: 0.0233
Epoch 7/15, Loss: 0.0217
Epoch 8/15, Loss: 0.0187
Epoch 9/15, Loss: 0.0178
Epoch 10/15, Loss: 0.0156
Epoch 11/15, Loss: 0.0153
Epoch 12/15, Loss: 0.0143
Epoch 13/15, Loss: 0.0116
Epoch 14/15, Loss: 0.0125
Epoch 15/15, Loss: 0.0118


In [23]:
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, preds = torch.max(outputs, 1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

In [24]:
accuracy = accuracy_score(all_labels, all_preds)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 99.40%


In [27]:
model_save_path = '/content/drive/My Drive/Drowsiness_model.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved at: {model_save_path}")

Model saved at: /content/drive/My Drive/Drowsiness_model.pth
